In [128]:
import pandas as pd

In [129]:
# 1. Найти месяц регистрации для каждого пользователя
# табличка users: user_registration_dt привести к месяцу (дату тоже оставляем)


# 2. Найти все сессии пользователей после регистрации
# из таблички sessions оставляем только сесси каждого пользователя, которые были после регистрации
# users join sessions по client_id - потом фильтруем session_start_ts (округлить до даты) >= user_registration_dt


# 3. Посчитать количество вернувшихся пользователей в каждый месяц после регистрации
# - если возврастные когорты, то месяц = 30 дней
# как считать: 
# - джоиним таблички users и sessions по client_id, 
# получаем табличку с сессиями, где значем дату регистрации
# - считаем возвраст когорты: для каждой сессии считаем, 
# сколько месяцев прошло с регистрации до этой сессии
# - если календарные когорты: приводим даты к месяцу и считаем разницу
# - если возврастные когорты: считаем разницу в днях и делим на 30
# агрегируем когорты по месяцам
# - группируем по месяцу регистраии, возрасту когорты (количество месяцев после регистрации)
# - агрегируем: считаем количество уникальных client_id (nunique())

# табличка с когортами:
# reg_month | cohort_age | n_returned_users

# Посчитать retention rate для каждого месяца после регистрации 
# - считаем количество пользователей, зарегистрированных в каждом месяце (агрегируем users)
# reg_month | total_users
# - джоиним агрегированную табличку с когортами
# reg_month | cohort_age | total_users | n_returned_users
# считаем retention rate: n_returned_users / total_users

# табличка с когортами:
# reg_month | cohort_age | total_users | n_returned_users | retention_rate

In [130]:
# 1. Найти месяц регистрации для каждого пользователя
# табличка users: user_registration_dt привести к месяцу (дату тоже оставляем)
users = pd.read_csv('/Users/artempak_macbook_air/Desktop/Code/python_learning/alex_python_for_pm/module_10_cohorts/self_tasks/data/users.csv', index_col='client_id')

users['user_registration_dt'] = pd.to_datetime(users['user_registration_dt'])
users['user_registration_day'] = users['user_registration_dt'].dt.to_period('D')
users['user_registration_month'] = users['user_registration_dt'].dt.to_period('M')
users.head(10)

,user_id,user_email,user_registration_dt,user_city,user_registration_day,user_registration_month
client_id,,,,,,
1120163048-362459827,1425,anilidic1822@outlook.com,2020-06-23,Екатеринбург,2020-06-23,2020-06
830333202-483914880,1429,bornagain1847@gmail.com,2020-08-15,Санкт-Петербург,2020-08-15,2020-08
28897314-329863990,1434,engluts2049@yahoo.com,2020-03-07,Новосибирск,2020-03-07,2020-03
1238119072-242362229,1436,hedgehog2019@live.com,2020-09-04,Москва,2020-09-04,2020-09
977210465-54763416,1439,chinook1914@yahoo.com,2020-06-29,Москва,2020-06-29,2020-06
1173856053-102092985,1443,snakemouths1913@outlook.com,2020-01-22,Москва,2020-01-22,2020-01
704990561-1223736114,1446,metaluminic1838@gmail.com,2020-02-14,Екатеринбург,2020-02-14,2020-02
921379771-1264690045,1451,guidage1946@yahoo.com,2020-07-01,Москва,2020-07-01,2020-07
880071473-301964486,1453,carambole2027@outlook.com,2020-11-23,Москва,2020-11-23,2020-11


In [131]:

# 2. Найти все сессии пользователей после регистрации
# из таблички sessions оставляем только сесси каждого пользователя, которые были после регистрации
# users join sessions по client_id - потом фильтруем session_start_ts (округлить до даты) >= user_registration_dt


# загружаю сессии
sessions = pd.read_csv('/Users/artempak_macbook_air/Desktop/Code/python_learning/alex_python_for_pm/module_10_cohorts/self_tasks/data/sessions.csv', index_col='client_id')

# делаю корректные даты сессий
sessions['session_start_ts'] = pd.to_datetime(sessions['session_start_ts'])
sessions['session_start_day'] = sessions['session_start_ts'].dt.to_period('D')
sessions['session_start_month'] = sessions['session_start_ts'].dt.to_period('M')
sessions

# джоиню
total_sessions = users.merge(sessions, how='inner', on='client_id')
total_sessions # 2762 строк

# фильтрую до даты регистрации до дней все строки
total_sessions = total_sessions.query('session_start_day >= user_registration_day')


# 3. Посчитать количество вернувшихся пользователей в каждый месяц после регистрации
#перевожу обратно в datetime
total_sessions['user_registration_day'] = total_sessions['user_registration_day'].dt.to_timestamp()
total_sessions['session_start_day'] = total_sessions['session_start_day'].dt.to_timestamp()

# - считаем возвраст когорты: для каждой сессии считаем, 
# сколько месяцев прошло с регистрации до этой сессии
total_sessions['cohort_day'] = (total_sessions['session_start_day'] - total_sessions['user_registration_day']).dt.days
total_sessions['cohort_month'] = total_sessions['cohort_day'] // 30
total_sessions


,user_id,user_email,user_registration_dt,user_city,user_registration_day,user_registration_month,session_id,session_start_ts,session_end_ts,session_duration,utm_source,utm_campaign,utm_medium,utm_term,utm_content,session_start_day,session_start_month,cohort_day,cohort_month
client_id,,,,,,,,,,,,,,,,,,,
1120163048-362459827,1425,anilidic1822@outlook.com,2020-06-23,Екатеринбург,2020-06-23,2020-06,19d51842-5ae7-11eb-b0b2-acde48001122,2020-06-23 10:54:25.161018,2020-06-23 10:55:18.223054,53,yandex_direct,gnrl|953233,gnrl,NaN,NaN,2020-06-23,2020-06,0,0
830333202-483914880,1429,bornagain1847@gmail.com,2020-08-15,Санкт-Петербург,2020-08-15,2020-08,19d53cbe-5ae7-11eb-b0b2-acde48001122,2020-08-15 02:51:04.403298,2020-08-15 02:52:21.465337,77,NaN,NaN,NaN,NaN,NaN,2020-08-15,2020-08,0,0
830333202-483914880,1429,bornagain1847@gmail.com,2020-08-15,Санкт-Петербург,2020-08-15,2020-08,19d5445c-5ae7-11eb-b0b2-acde48001122,2020-08-15 13:34:55.456394,2020-08-15 13:36:04.543422,69,fb,lal|772023,lal,NaN,NaN,2020-08-15,2020-08,0,0
28897314-329863990,1434,engluts2049@yahoo.com,2020-03-07,Новосибирск,2020-03-07,2020-03,19d54c7c-5ae7-11eb-b0b2-acde48001122,2020-03-07 09:06:53.278368,2020-03-07 09:07:41.280442,48,NaN,NaN,NaN,NaN,NaN,2020-03-07,2020-03,0,0
1238119072-242362229,1436,hedgehog2019@live.com,2020-09-04,Москва,2020-09-04,2020-09,19d5691e-5ae7-11eb-b0b2-acde48001122,2020-09-04 06:18:35.159181,2020-09-04 06:19:31.195269,56,NaN,NaN,NaN,NaN,NaN,2020-09-04,2020-09,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817367844-160697484,3125,abridged1849@live.com,2020-07-18,Казань,2020-07-18,2020-07,1a7a7ec2-5ae7-11eb-b0b2-acde48001122,2020-07-18 16:47:51.159163,2020-07-18 16:48:55.165244,64,google_ads,lal|487809,lal,NaN,NaN,2020-07-18,2020-07,0,0
817367844-160697484,3125,abridged1849@live.com,2020-07-18,Казань,2020-07-18,2020-07,1a7a89b2-5ae7-11eb-b0b2-acde48001122,2020-07-21 11:11:06.175213,2020-07-21 11:12:05.250302,59,yandex_direct,ret|574813,ret,NaN,NaN,2020-07-21,2020-07,3,0
817367844-160697484,3125,abridged1849@live.com,2020-07-18,Казань,2020-07-18,2020-07,1a7a8b38-5ae7-11eb-b0b2-acde48001122,2020-07-22 02:13:37.225287,2020-07-22 02:14:42.315349,65,yandex_direct,gnrl|326308,gnrl,NaN,NaN,2020-07-22,2020-07,4,0


In [ ]:
# агрегируем когорты по месяцам
# - группируем по месяцу регистраии, возрасту когорты (количество месяцев после регистрации)
cohort_agg = total_sessions.groupby(['user_registration_month','cohort_month'])

# - агрегируем: считаем количество уникальных client_id (nunique())
cohort_agg = cohort_agg.nunique('client_id')['user_id']